## **Preparing cnn file and dataset**

In [1]:
#!pip install -q tensorflow-model-optimization

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Loading files
import pickle

keras_file = '/content/drive/My Drive/Colab Notebooks/Capstone/testing/Outdoor_Living/outdoor_living_256.h5'
X = pickle.load(open("/content/drive/My Drive/Colab Notebooks/Capstone/testing/Outdoor_Living/X_Outdoor_Living.pickle", "rb"))
y = pickle.load(open("/content/drive/My Drive/Colab Notebooks/Capstone/testing/Outdoor_Living/y_Outdoor_Living.pickle", "rb"))
categories = pickle.load(open("/content/drive/My Drive/Colab Notebooks/Capstone/testing/Outdoor_Living/Outdoor_Living_categories.pickle", "rb"))

In [4]:
# Prepare dataset, we need to fit this to new pruned model.
# Accuracy should be the same or higher

from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
num_classes = len(categories)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2660, 256, 256, 1)
(2660, 38)
(1140, 256, 256, 1)
(1140, 38)


## **Building prune model**

In [5]:
# Build and compile new pruned model
import numpy as np
import tensorflow as tf
from tensorflow_model_optimization.sparsity import keras as sparsity

# Load the serialized model
loaded_model = tf.keras.models.load_model(keras_file)

epochs = 4
# end_step = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * epochs
# print(end_step)

new_pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                   final_sparsity=0.90,
                                                   begin_step=0,
                                                   end_step=5000,
                                                   frequency=100)
}

new_pruned_model = sparsity.prune_low_magnitude(loaded_model, **new_pruning_params)
new_pruned_model.summary()

new_pruned_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py:2191: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_conv2d ( (None, 256, 256, 32)      610       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 128, 128, 32)      1         
_________________________________________________________________
prune_low_magnitude_dropout  (None, 128, 128, 32)      1         
_________________________________________________________________
prune_low_magnitude_conv2d_1 (None, 128, 128, 64)      36930     
_________________________________________________________________
prune_low_magnitude_max_pool (None, 64, 64, 64)        1         
_________________________________________________________________
prune_low_magnitude_dropout_ (None, 64, 64, 64)        1         
_________________________________________________________________
prune_low_magnitude_flatten  (None, 262144)            1

In [6]:
logdir = '/content/drive/My Drive/Colab Notebooks/Capstone/testing/pruning_log'

callbacks = [
    sparsity.UpdatePruningStep(),
    sparsity.PruningSummaries(log_dir=logdir, profile_batch=0)
]

new_pruned_model.fit(X_train, y_train,
          epochs=10,
          verbose=1,
          callbacks=callbacks,
          validation_data=(X_test, y_test))

score = new_pruned_model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
84/84 [==============================] - 48s 161ms/step - loss: 0.3652 - accuracy: 0.9500 - val_loss: 2.4603 - val_accuracy: 0.6474
Epoch 2/10
84/84 [==============================] - 15s 171ms/step - loss: 0.3086 - accuracy: 0.9391 - val_loss: 2.2907 - val_accuracy: 0.6430
Epoch 3/10
84/84 [==============================] - 15s 171ms/step - loss: 0.2521 - accuracy: 0.9496 - val_loss: 2.3431 - val_accuracy: 0.6377
Epoch 4/10
84/84 [==============================] - 15s 171ms/step - loss: 0.2031 - accuracy: 0.9613 - val_loss: 2.3654 - val_accuracy: 0.6386
Epoch 5/10
84/84 [==============================] - 15s 171ms/step - loss: 0.1746 - accuracy: 0.9613 - val_loss: 2.3495 - val_accuracy: 0.6412
Epoch 6/10
84/84 [==============================] - 15s 171ms/step - loss: 0.1872 - accuracy: 0.9605 - val_loss: 2.3782 - val_accuracy: 0.6123
Epoc

In [7]:
final_model = sparsity.strip_pruning(new_pruned_model)
final_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 262144)            0

## **Saving pruned model**

In [8]:
new_pruned_keras_file = '/content/drive/My Drive/Colab Notebooks/Capstone/testing/Outdoor_Living/outdoor_living_pruned_256.h5'
print("Saving pruned model to: ", new_pruned_keras_file)
tf.keras.models.save_model(final_model, new_pruned_keras_file, include_optimizer=False)

Saving pruned model to:  /content/drive/My Drive/Colab Notebooks/Capstone/testing/Outdoor_Living/outdoor_living_pruned_256.h5


## **Reducing pruned model to x5 through zip3**

In [9]:
import tempfile
import zipfile
import os

# Zip the .h5 model file
zip3 = '/content/drive/My Drive/Colab Notebooks/Capstone/testing/Outdoor_Living/outdoor_living_pruned_256.h5.zip3'
with zipfile.ZipFile(zip3, "w", compression=zipfile.ZIP_DEFLATED) as f:
    f.write(new_pruned_keras_file)
print(
    "Size of the pruned model before compression: %.2f Mb"
    % (os.path.getsize(new_pruned_keras_file) / float(2 ** 20))
)
print(
    "Size of the pruned model after compression: %.2f Mb"
    % (os.path.getsize(zip3) / float(2 ** 20))
)

Size of the pruned model before compression: 1024.25 Mb
Size of the pruned model after compression: 432.22 Mb
